In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
def get_net(input_shape, load_weight_path=None, features=False, mal=False):
    width = 64
    inputs = Input(shape=(1, 32, 32, 32), name="input_1")
    x = inputs
    x = AveragePooling3D(pool_size=(2, 1, 1), strides=(2, 1, 1), border_mode="same")(x)
    x = Convolution3D(width, 3, 3, 3, activation='relu', border_mode='same', name='conv1', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), border_mode='valid', name='pool1')(x)

    # 2nd layer group
    x = Convolution3D(width*2, 3, 3, 3, activation='relu', border_mode='same', name='conv2', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool2')(x)
    x = Dropout(p=0.3)(x)

    # 3rd layer group
    x = Convolution3D(width*4, 3, 3, 3, activation='relu', border_mode='same', name='conv3a', subsample=(1, 1, 1))(x)
    x = Convolution3D(width*4, 3, 3, 3, activation='relu', border_mode='same', name='conv3b', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool3')(x)
    x = Dropout(p=0.4)(x)

    # 4th layer group
    x = Convolution3D(width*8, 3, 3, 3, activation='relu', border_mode='same', name='conv4a', subsample=(1, 1, 1))(x)
    x = Convolution3D(width*8, 3, 3, 3, activation='relu', border_mode='same', name='conv4b', subsample=(1, 1, 1),)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool4')(x)
    x = Dropout(p=0.5)(x)
       
    last64 = Convolution3D(64, 2, 2, 2, activation="relu", name="last_64")(x)
    out_class = Convolution3D(1, 1, 1, 1, activation="softmax", name="out_class_last")(last64)
    out_class = Flatten(name="out_class")(x)
    
    out_class = Dense(2)(out_class) 
    out_class = Activation('softmax')(out_class)
    
    model = Model(input=inputs, output=out_class)
    model.compile(optimizer=SGD(lr=1e-3, momentum=0.9, nesterov=True), loss='categorical_crossentropy', metrics=['accuracy'])
    #model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])   
    return model

In [3]:
output_path = PATH['cls_train_cube_30']
output_true = PATH['cls_train_cube_30_true']
output_false = PATH['cls_train_cube_30_false']
model_paths = PATH['model_paths']
model_final = PATH['model_final']

In [4]:
def get_dirfiles(dir):
    file_list = []
    subset_path = os.listdir(dir)
    for _ in range(len(subset_path)):
        if subset_path[_] != '.DS_Store':
            file_list.append(dir + subset_path[_])
    return file_list

def train_generator(output_true,output_false):
    train_nb = 21000    
    file_list_true = get_dirfiles(output_true)
    file_list_false = get_dirfiles(output_false)        
    file_list_true = np.random.choice(file_list_true, train_nb)
    file_list_false = np.random.choice(file_list_false, train_nb)    
    nb_true = len(file_list_true) + len(file_list_false)    
    sample = np.zeros([nb_true,32, 32, 32])
    labels = np.zeros([nb_true,2])
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels


def fenlei_fit(name, load_check = False,batch_size=2, epochs=100,check_name = None):
    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 8, verbose = 1),
                 ModelCheckpoint((model_paths + '{}.h5').format(name),
                                 monitor='val_loss',
                                 verbose = 0,
                                 save_best_only = True)]
    if load_check:
        check_model = (model_paths + '{}.h5').format(check_name)
        model = load_model(check_model)
    else:
        #model = classifier((1, 32, 32, 32),128,(3, 3, 3), (2, 2, 2))
        model = get_net((1, 32, 32, 32))
    x,y = train_generator(output_true,output_false)
    model.fit(x,y, batch_size=batch_size, epochs=epochs,
              validation_split=0.2,verbose=1, callbacks=callbacks, shuffle=True) 
    return model

In [5]:
fenlei_fit('Fenge_32_32_32_0630', load_check = True, batch_size=8, epochs=200, check_name = 'Fenge_32_32_32_0630')

100%|██████████| 21000/21000 [06:15<00:00, 55.88it/s]


Train on 33600 samples, validate on 8400 samples
Epoch 1/200
33600/33600 [==============================] - 645s - loss: 0.1877 - acc: 0.9242 - val_loss: 0.1784 - val_acc: 0.9299
Epoch 2/200
33600/33600 [==============================] - 588s - loss: 0.1008 - acc: 0.9607 - val_loss: 0.1504 - val_acc: 0.9452
Epoch 3/200
33600/33600 [==============================] - 588s - loss: 0.0668 - acc: 0.9758 - val_loss: 0.0871 - val_acc: 0.9704
Epoch 4/200
33600/33600 [==============================] - 588s - loss: 0.0455 - acc: 0.9830 - val_loss: 0.0408 - val_acc: 0.9873
Epoch 5/200
33600/33600 [==============================] - 586s - loss: 0.0372 - acc: 0.9867 - val_loss: 0.0429 - val_acc: 0.9855
Epoch 6/200
33600/33600 [==============================] - 590s - loss: 0.0245 - acc: 0.9912 - val_loss: 0.0233 - val_acc: 0.9927
Epoch 7/200
33600/33600 [==============================] - 589s - loss: 0.0215 - acc: 0.9926 - val_loss: 0.0411 - val_acc: 0.9877
Epoch 8/200
33600/33600 [================

In [6]:
file_list_true = get_dirfiles(output_true)
file_list_false = get_dirfiles(output_false)

In [7]:
#model_pred = classifier((1, 36, 36, 36), (3, 3, 3), (2, 2, 2))
model_pred = load_model(model_paths + 'Fenge_32_32_32_0630.h5')

In [8]:
file_list_true = np.random.choice(file_list_true, 1000)
file_list_false = np.random.choice(file_list_false, 1000)

In [9]:
cc = []
for i in file_list_false[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0
for i in cc:
    if i[0][0] > 0.9:
        count += 1
print count*1.0/len(cc)

0.985


In [10]:
cc

[array([[  9.99971509e-01,   2.85243314e-05]], dtype=float32),
 array([[  9.99942064e-01,   5.79487423e-05]], dtype=float32),
 array([[  9.99999762e-01,   1.81221026e-07]], dtype=float32),
 array([[  1.00000000e+00,   3.50741303e-08]], dtype=float32),
 array([[  9.99999881e-01,   1.14737539e-07]], dtype=float32),
 array([[  9.99999285e-01,   6.92984372e-07]], dtype=float32),
 array([[  9.99999881e-01,   6.63145556e-08]], dtype=float32),
 array([[  9.99981642e-01,   1.83575776e-05]], dtype=float32),
 array([[  9.99928474e-01,   7.15142887e-05]], dtype=float32),
 array([[  9.99996543e-01,   3.45003514e-06]], dtype=float32),
 array([[  9.99999642e-01,   3.71042916e-07]], dtype=float32),
 array([[  9.99998212e-01,   1.76442734e-06]], dtype=float32),
 array([[  9.99763429e-01,   2.36531268e-04]], dtype=float32),
 array([[  9.99977469e-01,   2.25615331e-05]], dtype=float32),
 array([[  1.00000000e+00,   5.35888738e-08]], dtype=float32),
 array([[ 0.42247301,  0.57752699]], dtype=float32),
 a

In [11]:
cc = []
for i in file_list_true[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0


In [12]:
for i in cc:
    if i[0][1] > 0.9:
        count += 1
print count*1.0/len(cc)

0.935


In [13]:
cc

[array([[  3.58085458e-06,   9.99996424e-01]], dtype=float32),
 array([[  7.12353676e-10,   1.00000000e+00]], dtype=float32),
 array([[ 0.00296474,  0.99703526]], dtype=float32),
 array([[  7.77415233e-04,   9.99222517e-01]], dtype=float32),
 array([[  8.67649396e-06,   9.99991298e-01]], dtype=float32),
 array([[  1.67736331e-08,   1.00000000e+00]], dtype=float32),
 array([[  5.79157262e-04,   9.99420881e-01]], dtype=float32),
 array([[  7.37518712e-05,   9.99926209e-01]], dtype=float32),
 array([[  1.60855561e-05,   9.99983907e-01]], dtype=float32),
 array([[  2.02465344e-05,   9.99979734e-01]], dtype=float32),
 array([[  3.06235634e-05,   9.99969363e-01]], dtype=float32),
 array([[  7.23556950e-05,   9.99927640e-01]], dtype=float32),
 array([[ 0.01534634,  0.98465365]], dtype=float32),
 array([[ 0.04402194,  0.95597804]], dtype=float32),
 array([[  5.12559791e-06,   9.99994874e-01]], dtype=float32),
 array([[  5.12203587e-06,   9.99994874e-01]], dtype=float32),
 array([[  5.03880067e